# mab_neut_ic50
Make neut curve plots with altair for all six monoclonal antibodies used for mapping escape in Nipah RBP
- Written by Brendan Larsen

In [ ]:
# this cell is tagged as parameters for `papermill` parameterization
altair_config = None
nipah_config = None
mab_neuts = None
mab_neuts_plot = None
mAb_neuts_table = None

In [ ]:
import warnings
import math

from IPython.display import display, HTML, SVG

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import neutcurve
from neutcurve.colorschemes import CBPALETTE
from neutcurve.colorschemes import CBMARKERS
import scipy.stats

from plotnine import *

import yaml

import altair as alt
import re
import os

print(f"Using `neutcurve` version {neutcurve.__version__}")

In [ ]:
# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

if (
    os.getcwd()
    == "/fh/fast/bloom_j/computational_notebooks/blarsen/2023/Nipah_Malaysia_RBP_DMS/"
):
    pass
    print("Already in correct directory")
else:
    os.chdir(
        "/fh/fast/bloom_j/computational_notebooks/blarsen/2023/Nipah_Malaysia_RBP_DMS/"
    )
    print("Setup in correct directory")

### For running interactively

In [ ]:
if mab_neuts_plot is None:
    altair_config = "data/custom_analyses_data/theme.py"
    nipah_config = "nipah_config.yaml"
    mab_neuts = "data/custom_analyses_data/experimental_data/NiV_neut_all_mabs.csv"

### Read in config files

In [ ]:
if altair_config:
    with open(altair_config, "r") as file:
        exec(file.read())

with open(nipah_config) as f:
    config = yaml.safe_load(f)

In [ ]:
# load data
df = pd.read_csv(mab_neuts)

In [ ]:
fits = neutcurve.curvefits.CurveFits(
    data=df,
    fixbottom=0,
)
fitParams = fits.fitParams(ics=[50, 90, 95, 97, 98, 99])
fitParams["ic50_ng"] = fitParams["ic50"] * 1000  # Convert from ug to ng
fitParams["ic99_ng"] = fitParams["ic99"] * 1000  # Convert from ug to ng
display(fitParams)
fit_df = fitParams[["serum", "ic50_ng"]].round(1)
fit_df = fit_df.loc[0:5]
fit_df.rename(columns={"serum": "Antibody", "ic50_ng": "ic50"}, inplace=True)
# Set 'Antibody' as the index
fit_df.set_index("Antibody", inplace=True)

# Define the new order you want; it must be a list of the index labels
new_order = ["m102.4", "HENV-26", "HENV-117", "HENV-103", "HENV-32", "nAH1.3"]

# Reindex the DataFrame with the new order
fit_df = fit_df.loc[new_order].reset_index()
fit_df["Epitope"] = [
    "Receptor Face",
    "Receptor Face",
    "Receptor Face",
    "Dimerization",
    "Dimerization",
    "Distal Head",
]
display(fit_df)
# Export IC50s for making a table
if mab_neuts_plot is not None:
    fit_df.to_csv(mAb_neuts_table, index=False)

In [ ]:
def extract_dataframe_from_neutcurve(serum, viruses, replicate="average"):
    curves = []
    # Loop over each virus type and retrieve the curve
    for sera in serum:
        curve = fits.getCurve(serum=sera, virus=viruses, replicate=replicate)
        df = curve.dataframe()
        df["antibody"] = sera
        curves.append(df)

    # Concatenate all the dataframes into one
    curve = pd.concat(curves, axis=0)
    curve["upper"] = curve["measurement"] + curve["stderr"]
    curve["lower"] = curve["measurement"] - curve["stderr"]

    #make neutralization fits
    curve['neut_fit'] = 1- curve['fit'] 
    curve['neut_measure'] = 1- curve['measurement'] 
    curve['neut_measure_upper'] = curve['neut_measure'] + curve['stderr']
    curve['neut_measure_lower'] = curve['neut_measure'] - curve['stderr']
    return curve


# Example usage:
serum = ["nAH1.3", "m102.4", "HENV-26", "HENV-32", "HENV-103", "HENV-117"]
viruses = "NiV"
curve = extract_dataframe_from_neutcurve(serum, viruses)
display(curve.head(3))

### Make altair plot of neut curves. Need to use three separate graphs. One for points, one for line, and one for stderr.

In [ ]:
def plot_neut():
    custom_order = ['m102.4','HENV-26','HENV-117','HENV-103','HENV-32','nAH1.3']
    cat10_colors = [
        "#4E79A5",
        "#F18F3B",
        "#E0585B",
        "#77B7B2",
        "#5AA155",
        "#EDC958",
        "#AF7AA0",
        "#FE9EA8",
        "#9C7561",
        "#BAB0AC",
    ]
    chart = (
        alt.Chart(curve)
        .mark_line(size=1.5, opacity=1)
        .encode(
            x=alt.X(
                "concentration:Q",
                scale=alt.Scale(type="log"),
                axis=alt.Axis(format=".0e"),
                title="Concentration (μg/mL)",
            ),
            y=alt.Y("fit:Q", title="Fraction Infectivity"),
            color=alt.Color(
                "antibody", title="Antibody", sort=custom_order,scale=alt.Scale(range=cat10_colors)
            ),
        )
        .properties(
            height=config["neut_curve_height"],
            width=config["neut_curve_width"],
        )
    )

    circle = (
        alt.Chart(curve)
        .mark_circle(size=40, opacity=1)
        .encode(
            x=alt.X(
                "concentration",
                scale=alt.Scale(type="log"),
                axis=alt.Axis(format=".0e"),
                title="Concentration (μg/mL)",
            ),
            y=alt.Y("measurement:Q", title="Fraction Infectivity"),
            color=alt.Color(
                "antibody", title="Antibody",sort=custom_order,scale=alt.Scale(range=cat10_colors)
            ),
        )
        .properties(
            height=config["neut_curve_height"],
            width=config["neut_curve_width"],
        )
    )

    error = (
        alt.Chart(curve)
        .mark_errorbar(opacity=1)
        .encode(
            x="concentration",
            y=alt.Y("lower", title="Fraction Infectivity", axis=alt.Axis(tickCount=3)),
            y2="upper",
            color=alt.Color("antibody",sort=custom_order,scale=alt.Scale(range=cat10_colors))
        )
    )
    plot = chart + circle + error
    return plot


neut_chart = plot_neut()
neut_chart.display()
if mab_neuts_plot is not None:
    neut_chart.save(mab_neuts_plot)

In [ ]:
ic50_test_df = pd.read_csv('data/custom_analyses_data/experimental_data/ic50_comparisons.csv')
display(ic50_test_df)

In [ ]:
custom_order = ['m102.4','HENV-26','HENV-117','HENV-103','HENV-32','nAH1.3']
cat10_colors = [
    "#4E79A5",
    "#F18F3B",
    "#E0585B",
    "#77B7B2",
    "#5AA155",
    "#EDC958",
    "#AF7AA0",
    "#FE9EA8",
    "#9C7561",
    "#BAB0AC",
]
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
        ic50_test_df["ic50"], ic50_test_df["DMS_ic50"]
)
display(r_value)
circle = (
    alt.Chart(ic50_test_df)
    .mark_circle(size=40, opacity=1)
    .encode(
        x=alt.X(
            "ic50",
            #scale=alt.Scale(type="log"),
            #axis=alt.Axis(format=".0e"),
            title="Experimental IC50",
        ),
        y=alt.Y("DMS_ic50:Q", title="DMS IC50"),
        color=alt.Color(
            "antibody", title="Antibody",sort=custom_order,scale=alt.Scale(range=cat10_colors)
        ),
    )
    .properties(
        height=200,
        width=200,
    )
)
text = (
        alt.Chart(
            {
                "values": [
                    {
                        "x": float(ic50_test_df["ic50"].min()),
                        "y": float(ic50_test_df["DMS_ic50"].max()),
                        "text": f"r = {r_value:.2f}",
                    }
                ]
            }
        )
        .mark_text(align="left", baseline="top", dx=5)  # Adjust this for position
        .encode(x=alt.X("x:Q"), y=alt.Y("y:Q"), text="text:N")
)
final_chart = circle + text
final_chart

In [ ]:
def plot_neut():
    custom_order = ['m102.4','HENV-26','HENV-117','HENV-103','HENV-32','nAH1.3']
    cat10_colors = [
        "#4E79A5",
        "#F18F3B",
        "#E0585B",
        "#77B7B2",
        "#5AA155",
        "#EDC958",
        "#AF7AA0",
        "#FE9EA8",
        "#9C7561",
        "#BAB0AC",
    ]
    chart = (
        alt.Chart(curve)
        .mark_line(size=1.5, opacity=1)
        .encode(
            x=alt.X(
                "concentration:Q",
                scale=alt.Scale(type="log"),
                axis=alt.Axis(format=".0e"),
                title="Concentration (μg/mL)",
            ),
            y=alt.Y("neut_fit:Q", title="Neutralization"),
            color=alt.Color(
                "antibody", title="Antibody", sort=custom_order,scale=alt.Scale(range=cat10_colors)
            ),
        )
        .properties(
            height=config["neut_curve_height"],
            width=config["neut_curve_width"],
        )
    )

    circle = (
        alt.Chart(curve)
        .mark_point(size=50, opacity=1)
        .encode(
            x=alt.X(
                "concentration",
                scale=alt.Scale(type="log"),
                axis=alt.Axis(format=".0e"),
                title="Concentration (μg/mL)",
            ),
            y=alt.Y("neut_measure:Q", title="Neutralization"),
            color=alt.Color(
                "antibody", title="Antibody",sort=custom_order,scale=alt.Scale(range=cat10_colors)
            ),
            #shape='antibody',
        )
        .properties(
            height=config["neut_curve_height"],
            width=config["neut_curve_width"],
        )
    )

    error = (
        alt.Chart(curve)
        .mark_errorbar(opacity=1,size=50)
        .encode(
            x="concentration",
            y=alt.Y("neut_measure_lower", title="Neutralization", axis=alt.Axis(tickCount=3)),
            y2="neut_measure_upper",
            color=alt.Color("antibody",sort=custom_order,scale=alt.Scale(range=cat10_colors))
        )
    )
    plot = chart + circle + error
    return plot


neut_chart = plot_neut()
neut_chart.display()
#if mab_neuts_plot is not None:
    #neut_chart.save(mab_neuts_plot)